# The associated data is in the file candy.csv. Develop the following models using maximum likelihood estimation (MLE):
## a. the Poisson model,
## b. the NBD model,
## c. the Zero-Inflated NBD model, and
## d. Finite Mixture models for 2, 3, and 4 segments.

In [1]:
import os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson
from scipy.optimize import minimize

<hr style="border:2px solid black">
<b>Read in the data</b>
<hr style="border:2px solid black">

In [2]:
file_path = 'C:/users/mared/desktop/MIS 6386/Project 2/candy.csv'
candy = pd.read_csv('candy.csv')
candy.head()

,Packs,People
0,0,102
1,1,54
2,2,49
3,3,62
4,4,44


# Poisson Model

<hr style="border:2px solid black">
<b>Define the funtions needed</b><br>
<b>1. the log-likelihood</b>
<hr style="border:2px solid black">

In [3]:
def ll_poisson(params, packs, people):
    lam = params
    
    prob = []
    prob_t =[]
    #probSum = 0
    
# need to consider all periods
    pds = list(range(0, len(packs)))
    for i in pds:
            prob.append(poisson.pmf(packs[i],lam))
            prob_t.append(people[i]*np.log(prob[i]))
        
    return sum(prob_t)

<hr style="border:2px solid black">
<b>2. the negative of the log-likelihood</b>
<hr style="border:2px solid black">

In [4]:
def neg_ll_poisson(params, packs, people):
    return(-np.sum(ll_poisson(params, packs, people)))

<hr style="border:2px solid black">
<b>3. Specify the information needed for the optimizer</b>
<hr style="border:2px solid black">

In [5]:
packs = candy.Packs
people = candy.People
params = np.mean(packs)
print(params, ll_poisson(params, packs, people))
print('neg:', neg_ll_poisson(params, packs, people))


10.0 -2613.351647612551
neg: 2613.351647612551


<hr style="border:2px solid black">
<b>4. Call the optimizer</b>
<hr style="border:2px solid black">

In [6]:
result_poisson = minimize(
    neg_ll_poisson, 
    params,
    args=(packs, people))

<hr style="border:2px solid black">
<b>5. Review the result</b>
<hr style="border:2px solid black">

In [7]:
result_poisson

      fun: 1544.9963904489678
 hess_inv: array([[0.00814079]])
      jac: array([0.])
  message: 'Optimization terminated successfully.'
     nfev: 20
      nit: 7
     njev: 10
   status: 0
  success: True
        x: array([3.99122794])

In [8]:
print("Estimated Poisson Parameter (lambda):", result_poisson.x[0])
print("Maximum Log-Likelihood (Poisson Model):", -result_poisson.fun)

Estimated Poisson Parameter (lambda): 3.991227942685413
Maximum Log-Likelihood (Poisson Model): -1544.9963904489678


<b>The estimated parameter (λ) for the Poisson model is approximately 3.991, and the maximum value of the log-likelihood is approximately -1544.996.<b>

# NBD Model

<hr style="border:2px solid black">
<b>Define the funtions needed</b><br>
<b>1. the log-likelihood</b>
<hr style="border:2px solid black">

In [9]:
def log_likelihood_nbd(params, packs, people):
    shape_n, alpha = params
    
    prob = []
    prob_t =[]
    #probSum = 0
    
# need to consider all periods
    pds = list(range(0, len(packs)))
    for i in pds:
        if i == 0:
            prob.append((alpha/(alpha+1))**shape_n)
            prob_t.append(people[i]*np.log(prob[i]))
            #probSum += prob_t[i]
        else:
            prob.append(prob[i-1]*((shape_n)+(packs[i])-1)/(packs[i]*((alpha)+1)))
            prob_t.append(people[i]*np.log(prob[i]))
            #probSum += prob_t[i]
        
    return sum(prob_t)

<hr style="border:2px solid black">
<b>2. the negative of the log-likelihood</b>
<hr style="border:2px solid black">

In [10]:
def neg_log_likelihood_nbd(params, packs, people):
    return(-np.sum(log_likelihood_nbd(params, packs, people)))

<hr style="border:2px solid black">
<b>3. Specify the information needed for the optimizer</b>
<hr style="border:2px solid black">

In [11]:
packs = candy.Packs
people = candy.People
params = np.array((1,0.5))
print(log_likelihood_nbd(params, packs, people))
print('neg:', neg_log_likelihood_nbd(params, packs, people))


-1238.913700389517
neg: 1238.913700389517


<hr style="border:2px solid black">
<b>4. Call the optimizer</b>
<hr style="border:2px solid black">

In [12]:
result_nbd_final = minimize(
    neg_log_likelihood_nbd, 
    [1, 0.5],  # Inline initial guess
    args=(packs, people), 
    bounds=((1e-5, None), (1e-5, 1-1e-5))  # Inline bounds
)

<hr style="border:2px solid black">
<b>5. Review the result</b>
<hr style="border:2px solid black">

In [13]:
result_nbd_final

      fun: 1140.0237461861923
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.00079581, -0.00138698])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 36
      nit: 8
     njev: 12
   status: 0
  success: True
        x: array([0.99766195, 0.2499634 ])

In [14]:
print("Estimated NBD Parameters (shape, alpha):", result_nbd_final.x)
print("Maximum Log-Likelihood (NBD Model):", -result_nbd_final.fun)

Estimated NBD Parameters (shape, alpha): [0.99766195 0.2499634 ]
Maximum Log-Likelihood (NBD Model): -1140.0237461861923


<b>The estimated alpha parameter for the NBD model is approximately 0.998, the estimated shape parameter is 0.250 and the maximum value of the log-likelihood is approximately -1140.024.<b>

# Zero-Inflated NBD Model

<hr style="border:2px solid black">
<b>Define the funtions needed</b><br>
<b>1. the log-likelihood</b>
<hr style="border:2px solid black">

In [15]:
def log_likelihood_znbd(params, packs, people):
    shape_n, alpha, pi = params
    
    prob = []
    prob_t =[]
    nbd = []
    #probSum = 0
    
# need to consider all periods
    pds = list(range(0, len(packs)))
    for i in pds:
        if i == 0:
            nbd.append((alpha/(alpha+1))**shape_n)
            prob.append(pi+(1-pi)*nbd[i])
            prob_t.append(people[i]*np.log(prob[i]))
            #probSum += prob_t[i]
        else:
            nbd.append(nbd[i-1]*((shape_n)+(packs[i])-1)/(packs[i]*((alpha)+1)))
            prob.append((1-pi)*nbd[i])
            prob_t.append(people[i]*np.log(prob[i]))
            #probSum += prob_t[i]
        
    return sum(prob_t)

<hr style="border:2px solid black">
<b>2. the negative of the log-likelihood</b>
<hr style="border:2px solid black">

In [16]:
def neg_log_likelihood_znbd(params, packs, people):
    return(-np.sum(log_likelihood_znbd(params, packs, people)))

<hr style="border:2px solid black">
<b>3. Specify the information needed for the optimizer</b>
<hr style="border:2px solid black">

In [17]:
packs = candy.Packs
people = candy.People
params = np.array((1,0.5,0.5))
print(log_likelihood_znbd(params, packs, people))
print('neg:', neg_log_likelihood_znbd(params, packs, people))

-1413.5867898906235
neg: 1413.5867898906235


<hr style="border:2px solid black">
<b>4. Call the optimizer</b>
<hr style="border:2px solid black">

In [18]:
result_znbd_final = minimize(
    neg_log_likelihood_znbd, 
    [1, 0.5,0.5],  # Inline initial guess
    args=(packs, people), 
    bounds=((1e-5, None), (1e-5, 1-1e-5),(1e-5, None))  # Inline bounds
)

<hr style="border:2px solid black">
<b>5. Review the result</b>
<hr style="border:2px solid black">

In [19]:
result_znbd_final

      fun: 1136.1656408318265
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([-9.09494707e-05,  6.59383659e-04,  1.36424205e-04])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 60
      nit: 14
     njev: 15
   status: 0
  success: True
        x: array([1.5039228 , 0.33418861, 0.11310451])

In [20]:
print("Estimated Zero-inflated NBD Parameters (shape, alpha, pi):", result_znbd_final.x)
print("Maximum Log-Likelihood (NBD Model):", -result_znbd_final.fun)

Estimated Zero-inflated NBD Parameters (shape, alpha, pi): [1.5039228  0.33418861 0.11310451]
Maximum Log-Likelihood (NBD Model): -1136.1656408318265


<b>The estimated shape parameter for the zero-inflated NBD model is approximately 1.504, the estimated alpha parameter is 0.334, pi value is 0.113 and the maximum value of the log-likelihood is approximately -1136.166.<b>

# Finite Mixture Models

## 2 Segment

<hr style="border:2px solid black">
<b>Define the funtions needed</b><br>
<b>1. the log-likelihood</b>
<hr style="border:2px solid black">

In [21]:
def log_likelihood_fmm2(params, packs, people):
    lam1, lam2, pi = params
    
    prob = []
    prob_t =[]
    seg1 = []
    seg2 = []
    E = []
    #probSum = 0
    
# need to consider all periods
    pds = list(range(0, len(packs)))
    for i in pds:
        if i == 0:
            seg1.append(poisson.pmf(packs[i],lam1))
            seg2.append(poisson.pmf(packs[i],lam2))
            prob.append(pi*seg1[i]+(1-pi)*seg2[i])
            prob_t.append(people[i]*np.log(prob[i]))
            E.append(prob[i]*people.sum())
            #probSum += prob_t[i]
        else:
            seg1.append(poisson.pmf(packs[i],lam1))
            seg2.append(poisson.pmf(packs[i],lam2))
            prob.append(pi*seg1[i]+(1-pi)*seg2[i])
            prob_t.append(people[i]*np.log(prob[i]))
            E.append(prob[i]*people.sum())
            #probSum += prob_t[i]
        
    return sum(prob_t)

<hr style="border:2px solid black">
<b>2. the negative of the log-likelihood</b>
<hr style="border:2px solid black">

In [22]:
def neg_log_likelihood_fmm2(params, packs, people):
    return(-np.sum(log_likelihood_fmm2(params, packs, people)))

<hr style="border:2px solid black">
<b>3. Specify the information needed for the optimizer</b>
<hr style="border:2px solid black">

In [23]:
packs = candy.Packs
people = candy.People
params = np.array((1,0.5,0.5))
print(log_likelihood_fmm2(params, packs, people))
print('neg:', neg_log_likelihood_fmm2(params, packs, people))

-2849.197475099176
neg: 2849.197475099176


<hr style="border:2px solid black">
<b>4. Call the optimizer</b>
<hr style="border:2px solid black">

In [24]:
result_fmm2_final = minimize(
    neg_log_likelihood_fmm2, 
    [1, 0.5,0.5],  # Inline initial guess
    args=(packs, people), 
    bounds=((1e-5, None), (1e-5, None),(1e-5, 0.99999))  # Inline bounds
)

<hr style="border:2px solid black">
<b>5. Review the result</b>
<hr style="border:2px solid black">

In [25]:
result_fmm2_final

      fun: 1188.8328271728494
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 4.54747313e-05, -3.18323148e-04, -1.81898940e-04])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 68
      nit: 16
     njev: 17
   status: 0
  success: True
        x: array([9.1206904 , 1.80215205, 0.29911381])

In [26]:
print("Estimated 2 segment finite mixture model Parameters (lambda 1, lambda 2, pi):", result_fmm2_final.x)
print("Maximum Log-Likelihood (2 segment finite mixture Model):", -result_fmm2_final.fun)

Estimated 2 segment finite mixture model Parameters (lambda 1, lambda 2, pi): [9.1206904  1.80215205 0.29911381]
Maximum Log-Likelihood (2 segment finite mixture Model): -1188.8328271728494


## 3 Segment

<hr style="border:2px solid black">
<b>Define the funtions needed</b><br>
<b>1. the log-likelihood</b>
<hr style="border:2px solid black">

In [27]:
def log_likelihood_fmm3(params, packs, people):
    lam1, lam2, lam3, theta1, theta2, theta3 = params
    
    theta1=params[3]
    theta2=params[4]
    theta3=params[5]=0
    prob = []
    prob_t =[]
    seg1 = []
    seg2 = []
    seg3 = []
    E = []

    #probSum = 0
    
# need to consider all periods
    pds = list(range(0, len(packs)))
    for i in pds:
        if i == 0:
            seg1.append(poisson.pmf(packs[i],lam1))
            seg2.append(poisson.pmf(packs[i],lam2))
            seg3.append(poisson.pmf(packs[i],lam3))
                        
            prob.append(((np.exp(theta1))/(np.exp(theta1)+np.exp(theta2)+np.exp(theta3)))*seg1[i]+
                        ((np.exp(theta2))/(np.exp(theta1)+np.exp(theta2)+np.exp(theta3)))*seg2[i]+
                        ((np.exp(theta3))/(np.exp(theta1)+np.exp(theta2)+np.exp(theta3)))*seg3[i])
            prob_t.append(people[i]*np.log(prob[i]))
            E.append(prob[i]*people.sum())
            #probSum += prob_t[i]
        else:
            seg1.append(poisson.pmf(packs[i],lam1))
            seg2.append(poisson.pmf(packs[i],lam2))
            seg3.append(poisson.pmf(packs[i],lam3))
            
            prob.append(((np.exp(theta1))/(np.exp(theta1)+np.exp(theta2)+np.exp(theta3)))*seg1[i]+
                        ((np.exp(theta2))/(np.exp(theta1)+np.exp(theta2)+np.exp(theta3)))*seg2[i]+
                        ((np.exp(theta3))/(np.exp(theta1)+np.exp(theta2)+np.exp(theta3)))*seg3[i])
            prob_t.append(people[i]*np.log(prob[i]))
            E.append(prob[i]*people.sum())
            #probSum += prob_t[i]
        
    return sum(prob_t)

<hr style="border:2px solid black">
<b>2. the negative of the log-likelihood</b>
<hr style="border:2px solid black">

In [28]:
def neg_log_likelihood_fmm3(params, packs, people):
    return(-np.sum(log_likelihood_fmm3(params, packs, people)))

<hr style="border:2px solid black">
<b>3. Specify the information needed for the optimizer</b>
<hr style="border:2px solid black">

In [29]:
packs = candy.Packs
people = candy.People
params = np.array((0.2,2,7,1,2,0))
print(log_likelihood_fmm3(params, packs, people))
print('neg:', neg_log_likelihood_fmm3(params, packs, people))

-1275.4233903049492
neg: 1275.4233903049492


<hr style="border:2px solid black">
<b>4. Call the optimizer</b>
<hr style="border:2px solid black">

In [30]:
result_fmm3_final = minimize(
    neg_log_likelihood_fmm3, 
    [0.2,2,7,1,2,0],  # Inline initial guess
    args=(packs, people), 
    bounds=((1e-5,None),(1e-5,None),(1e-5,None),(1e-5,None),(1e-5,None),(None,None))  # Inline bounds
)

<hr style="border:2px solid black">
<b>5. Review the result</b>
<hr style="border:2px solid black">

In [31]:
result_fmm3_final

      fun: 1132.0429842574208
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 4.09272616e-04, -1.13686838e-04, -9.09494627e-05, -2.50111043e-04,
        1.36424206e-04,  0.00000000e+00])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 112
      nit: 14
     njev: 16
   status: 0
  success: True
        x: array([ 0.29054317,  3.48331344, 11.21579157,  0.43041889,  1.10485004,
        0.        ])

In [32]:
print("Estimated 3 segment finite mixture model Parameters (lambda 1, lambda 2, lamda 3, theta 1, theta 2, theta 3):", result_fmm3_final.x)
print("Maximum Log-Likelihood (2 segment finite mixture Model):", -result_fmm3_final.fun)

Estimated 3 segment finite mixture model Parameters (lambda 1, lambda 2, lamda 3, theta 1, theta 2, theta 3): [ 0.29054317  3.48331344 11.21579157  0.43041889  1.10485004  0.        ]
Maximum Log-Likelihood (2 segment finite mixture Model): -1132.0429842574208


## 4 Segment

<hr style="border:2px solid black">
<b>Define the funtions needed</b><br>
<b>1. the log-likelihood</b>
<hr style="border:2px solid black">

In [33]:
def log_likelihood_fmm4(params, packs, people):
    lam1, lam2, lam3, lam4, theta1, theta2, theta3, theta4 = params
    
    theta1=params[4]
    theta2=params[5]
    theta3=params[6]
    theta4=params[7]=0
    prob = []
    prob_t =[]
    seg1 = []
    seg2 = []
    seg3 = []
    seg4 = []
    E = []

    #probSum = 0
    
# need to consider all periods
    pds = list(range(0, len(packs)))
    for i in pds:
        if i == 0:
            seg1.append(poisson.pmf(packs[i],lam1))
            seg2.append(poisson.pmf(packs[i],lam2))
            seg3.append(poisson.pmf(packs[i],lam3))
            seg4.append(poisson.pmf(packs[i],lam4))
                        
            prob.append(((np.exp(theta1))/(np.exp(theta1)+np.exp(theta2)+np.exp(theta3)+np.exp(theta4)))*seg1[i]+
                        ((np.exp(theta2))/(np.exp(theta1)+np.exp(theta2)+np.exp(theta3)+np.exp(theta4)))*seg2[i]+
                        ((np.exp(theta3))/(np.exp(theta1)+np.exp(theta2)+np.exp(theta3)+np.exp(theta4)))*seg3[i]+
                        ((np.exp(theta4))/(np.exp(theta1)+np.exp(theta2)+np.exp(theta3)+np.exp(theta4)))*seg4[i])
            prob_t.append(people[i]*np.log(prob[i]))
            E.append(prob[i]*people.sum())
            #probSum += prob_t[i]
        else:
            seg1.append(poisson.pmf(packs[i],lam1))
            seg2.append(poisson.pmf(packs[i],lam2))
            seg3.append(poisson.pmf(packs[i],lam3))
            seg4.append(poisson.pmf(packs[i],lam4))
            
            prob.append(((np.exp(theta1))/(np.exp(theta1)+np.exp(theta2)+np.exp(theta3)+np.exp(theta4)))*seg1[i]+
                        ((np.exp(theta2))/(np.exp(theta1)+np.exp(theta2)+np.exp(theta3)+np.exp(theta4)))*seg2[i]+
                        ((np.exp(theta3))/(np.exp(theta1)+np.exp(theta2)+np.exp(theta3)+np.exp(theta4)))*seg3[i]+
                        ((np.exp(theta4))/(np.exp(theta1)+np.exp(theta2)+np.exp(theta3)+np.exp(theta4)))*seg4[i])
            prob_t.append(people[i]*np.log(prob[i]))
            E.append(prob[i]*people.sum())
            #probSum += prob_t[i]
        
    return sum(prob_t)

<hr style="border:2px solid black">
<b>2. the negative of the log-likelihood</b>
<hr style="border:2px solid black">

In [34]:
def neg_log_likelihood_fmm4(params, packs, people):
    return(-np.sum(log_likelihood_fmm4(params, packs, people)))

<hr style="border:2px solid black">
<b>3. Specify the information needed for the optimizer</b>
<hr style="border:2px solid black">

In [35]:
packs = candy.Packs
people = candy.People
params = np.array((3,0.2,7,13,2,1,0.5,0))
print(log_likelihood_fmm4(params, packs, people))
print('neg:', neg_log_likelihood_fmm4(params, packs, people))

-1134.7141082265887
neg: 1134.7141082265887


<hr style="border:2px solid black">
<b>4. Call the optimizer</b>
<hr style="border:2px solid black">

In [36]:
result_fmm4_final = minimize(
    neg_log_likelihood_fmm4, 
    [3,0.2,7,13,2,1,0.5,0],  # Inline initial guess
    args=(packs, people), 
    bounds=((1e-5,None),(1e-5,None),(1e-5,None),(1e-5,None),(1e-5,None),(1e-5,None),(1e-5,None),(None,None))  # Inline bounds
)

<hr style="border:2px solid black">
<b>5. Review the result</b>
<hr style="border:2px solid black">

In [37]:
result_fmm4_final

      fun: 1130.0705911485175
 hess_inv: <8x8 LbfgsInvHessProduct with dtype=float64>
      jac: array([-1.43245416e-03,  4.54747351e-04,  2.27373677e-04,  4.32009948e-04,
       -7.95807869e-04, -9.09494697e-05, -4.77484719e-04,  0.00000000e+00])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 243
      nit: 22
     njev: 27
   status: 0
  success: True
        x: array([ 3.00183067,  0.20471233,  7.41787252, 12.87254227,  1.59796451,
        0.87588531,  0.39785026,  0.        ])

In [38]:
print("Estimated 4 segment finite mixture model Parameters (lambda 1, lambda 2, lamda 3, lambda 4, theta 1, theta 2, theta 3, theta 4):", result_fmm4_final.x)
print("Maximum Log-Likelihood (4 segment finite mixture Model):", -result_fmm4_final.fun)

Estimated 4 segment finite mixture model Parameters (lambda 1, lambda 2, lamda 3, lambda 4, theta 1, theta 2, theta 3, theta 4): [ 3.00183067  0.20471233  7.41787252 12.87254227  1.59796451  0.87588531
  0.39785026  0.        ]
Maximum Log-Likelihood (4 segment finite mixture Model): -1130.0705911485175


# Evaluating the models developed using log-likelihood ratio, AIC, BIC

In [39]:
# AIC=−2×Log-Likelihood+2×Number of Parameters
# BIC=−2×Log-Likelihood+log(N)×Number of Parameters

# Log likelihood
poisson_LL = -result_poisson.fun
nbd_LL = -result_nbd_final.fun
zeroinflatednbd_LL = -result_znbd_final.fun
segment2finite_LL = -result_fmm2_final.fun
segment3finite_LL = -result_fmm3_final.fun
segment4finite_LL = -result_fmm4_final.fun

# Number of Parameters
poisson_len = len(result_poisson.x)
nbd_len = len(result_nbd_final.x)
zeroinflatednbd_len = len(result_znbd_final.x)
segment2finite_len = len(result_fmm2_final.x)
segment3finite_len = len(result_fmm3_final.x)
segment4finite_len = len(result_fmm4_final.x)

# AIC
AIC_poisson = (-2*poisson_LL) + (2*poisson_len)
AIC_nbd = (-2*nbd_LL) + (2*nbd_len)
AIC_zeroinflatednbd = (-2*zeroinflatednbd_LL) + (2*zeroinflatednbd_len)
AIC_segment2finite = (-2*segment2finite_LL) + (2*segment2finite_len)
AIC_segment3finite = (-2*segment3finite_LL) + (2*segment3finite_len)
AIC_segment4finite = (-2*segment4finite_LL) + (2*segment4finite_len)

#BIC
N = candy.shape[0]

BIC_poisson = (-2*poisson_LL) + (np.log(N)*poisson_len)
BIC_nbd = (-2*nbd_LL) + (np.log(N)*nbd_len)
BIC_zeroinflatednbd = (-2*zeroinflatednbd_LL) + (np.log(N)*zeroinflatednbd_len)
BIC_segment2finite = (-2*segment2finite_LL) + (np.log(N)*segment2finite_len)
BIC_segment3finite = (-2*segment3finite_LL) + (np.log(N)*segment3finite_len)
BIC_segment4finite = (-2*segment4finite_LL) + (np.log(N)*segment4finite_len)


print("Poisson Model (LL, AIC, BIC):", poisson_LL, AIC_poisson, BIC_poisson)
print("NBD Model (LL, AIC, BIC):", nbd_LL, AIC_nbd, BIC_nbd)
print("Zero-Inflated NBD Model (LL, AIC, BIC):", zeroinflatednbd_LL, AIC_zeroinflatednbd, BIC_zeroinflatednbd)
print("Finite Mixture Model - 2 Segment (LL, AIC, BIC):", segment2finite_LL, AIC_segment2finite, BIC_segment2finite)
print("Finite Mixture Model - 3 Segment (LL, AIC, BIC):", segment3finite_LL, AIC_segment3finite, BIC_segment3finite)
print("Finite Mixture Model - 4 Segment (LL, AIC, BIC):", segment4finite_LL, AIC_segment4finite, BIC_segment4finite)

Poisson Model (LL, AIC, BIC): -1544.9963904489678 3091.9927808979355 3093.037303335659
NBD Model (LL, AIC, BIC): -1140.0237461861923 2284.0474923723846 2286.1365372478313
Zero-Inflated NBD Model (LL, AIC, BIC): -1136.1656408318265 2278.331281663653 2281.4648489768233
Finite Mixture Model - 2 Segment (LL, AIC, BIC): -1188.8328271728494 2383.665654345699 2386.799221658869
Finite Mixture Model - 3 Segment (LL, AIC, BIC): -1132.0429842574208 2276.0859685148416 2282.353103141182
Finite Mixture Model - 4 Segment (LL, AIC, BIC): -1130.0705911485175 2276.141182297035 2284.497361798822


## •	The model with the highest log-likelihood is generally considered better as it maximizes the likelihood of the observed data.
## •	AIC and BIC are used for model comparison, with lower values indicating better-fitting models.

## Based on above Results:
## •	The Poisson model has the highest Log-Likelihood, indicating a better fit in terms of likelihood.
## •	The Zero-Inflated NBD model has a slightly better AIC and BIC compared to the NBD model.
## •	The Finite Mixture Models have varying segmentations, and their performance depends on the complexity needed.

## The differences could be due to the underlying distribution assumptions and the complexity of the models. Finite Mixture Models introduce segmentation, capturing different groups within the data.
